1. Персона — челоке/персонаж, упоминаемый в тексте (Например: Настя Ивлеева, Дядя Федор, Баста, Евгений Онегин). Персона - только имя человека/персонажа. Например, фраза "Руководитель отдела" не является персоной.

2. Локация — географические координаты, такие как страна, город, край, область, название пространства (Например: Москва, Россия, Казахстан, на Фестивальной улице, гостица Солнечная)

3. Дата — дата события (Например: 4 января, в 1992 году, в октябре)

4. Организация — организация, орган государственной власти, учреждение (Например: Яндекс, Министерство цифрового развития, Совет Федерации, ООО "Рога и Копыта")

5. Бренд - бренд (Например:  Samsung, Audi, Toyota, )

6. Модель - название модели (Например: Galaxy S10, RS6, Corolla)

7. Название проекта - название шоу, сериала, фильма, проекта организации (Например: Битва экстрасенсов, беременна в 16, Интерны, Лидеры России, Международная кооперация и экспорт)

8. Сезон - сезон/часть сериала (Например: третий, первый, 1)

9. Серия - серия сериала, передачи (Например: первая, заключительная, 4)

10. Лига - название спортивной лиги (Например: Чемпионат Европы-2024, Лалига, Английская премьер лига, Восточно-европейская хоккейная лига, чемпионате мира)

11. Команда - команда (спортивная, студенческая) (Например: ЦСКА, Manchester United, КАМАЗ-мастер)

12. Вид спорта - название спорта или подтипа (Например: футбол, керлинг, автоспорт, танго)

13. Видеоигра — название видеоигры (Например: GTA, Call of Duty, Майнкрафт)

## Создание среды проекта

In [ ]:
# pip install transformers evaluate  install accelerate seqeval

In [1]:
# import os
import json
import numpy as np
import pandas as pd
import razdel
from tqdm.autonotebook import tqdm

import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from datasets import Dataset, DatasetDict

from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import load_dataset, load_metric

# import logging

/tmp/ipykernel_76521/1682736822.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


---

## Preparing data

In [40]:
data = pd.read_csv("ner_data_train.csv")
data.head(5)

,video_info,entities
0,<НАЗВАНИЕ:> Агент 117: Из Африки с любовью — Р...,"{""label"":""локация""\,""offset"":26\,""length"":6\,""..."
1,<НАЗВАНИЕ:> Коленвал Инфинити Ку икс 56= 5.6 V...,"{""label"":""организация""\,""offset"":196\,""length""..."
2,<НАЗВАНИЕ:> ВЫЗОВ ДЕМОНА = Вызвал Серого Челов...,"{""label"":""название проекта""\,""offset"":12\,""len..."
3,<НАЗВАНИЕ:> Довоенная немецкая кирха в Калинин...,"{""label"":""не найдено""\,""offset"":162\,""length"":..."
4,"<НАЗВАНИЕ:> ""Спартаку"" помогли судьи? Локомоти...","{""label"":""команда""\,""offset"":13\,""length"":8\,""..."


In [41]:
# данные спарсены с Толоки, поэтому могут иметь проблемы с символами и их нужно избежать, 
# удалить лишние '\' например, преобразовать из str в список dict-ов
df = data.copy()
df['entities'] = df['entities'].apply(lambda l: l.replace('\,', ',')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: l.replace('\\\\', '\\')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: '[' + l + ']'if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: json.loads(l)if isinstance(l, str) else l)

df.head(5)

,video_info,entities
0,<НАЗВАНИЕ:> Агент 117: Из Африки с любовью — Р...,"[{'label': 'локация', 'offset': 26, 'length': ..."
1,<НАЗВАНИЕ:> Коленвал Инфинити Ку икс 56= 5.6 V...,"[{'label': 'организация', 'offset': 196, 'leng..."
2,<НАЗВАНИЕ:> ВЫЗОВ ДЕМОНА = Вызвал Серого Челов...,"[{'label': 'название проекта', 'offset': 12, '..."
3,<НАЗВАНИЕ:> Довоенная немецкая кирха в Калинин...,"[{'label': 'не найдено', 'offset': 162, 'lengt..."
4,"<НАЗВАНИЕ:> ""Спартаку"" помогли судьи? Локомоти...","[{'label': 'команда', 'offset': 13, 'length': ..."


In [42]:
# Теперь из наших данных нам нужно извлечь для каждого слова (токена) его
# тег (label) из разметки, чтобы потом предать в модель классификации токенов

def extract_labels(item):
    
    # воспользуемся удобным токенайзером из библиотеки razdel, 
    # она помимо разбиения на слова, сохраняет важные
    # для нас числа - начало и конец слова в токенах
    
    raw_toks = list(razdel.tokenize(item['video_info']))
    words = [tok.text for tok in raw_toks]
    # присвоим для начала каждому слову тег 'О' - тег, означающий отсутствие NER-а
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item['video_info'])
    # так как NER можем состаять из нескольких слов, то нам нужно сохранить эту инфорцию
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    labels = item['entities']
    if isinstance(labels, dict):
        labels = [labels]
    if labels is not None:
        for e in labels:
            if e['label'] != 'не найдено':
                e_words = sorted({idx for idx in char2word[e['offset']:e['offset']+e['length']] if idx is not None})
                if e_words:
                    word_labels[e_words[0]] = 'B-' + e['label']
                    for idx in e_words[1:]:
                        word_labels[idx] = 'I-' + e['label']
                else:
                    continue
            else:
                continue
        return {'tokens': words, 'tags': word_labels}
    else: return {'tokens': words, 'tags': word_labels}

In [43]:
# extract_labels(df.iloc[0])

### Making train ant test

In [44]:
ner_data = [extract_labels(item) for i, item in df.iterrows()]
ner_train, ner_test = train_test_split(ner_data, test_size=0.01, random_state=1)

In [45]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O',
 'B-Дата',
 'B-бренд',
 'B-вид спорта',
 'B-видеоигра',
 'B-команда',
 'B-лига',
 'B-локация',
 'B-модель',
 'B-название проекта',
 'B-организация',
 'B-персона',
 'B-сезон',
 'B-серия',
 'I-Дата',
 'I-бренд',
 'I-вид спорта',
 'I-видеоигра',
 'I-команда',
 'I-лига',
 'I-локация',
 'I-модель',
 'I-название проекта',
 'I-организация',
 'I-персона',
 'I-сезон',
 'I-серия']

In [46]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 6357
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 65
    })
})

---

## Модель

In [9]:
model_checkpoint = "xlm-roberta-large-finetuned-conll03-english"
batch_size = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, device=device)
device

device(type='cuda')

#### У Bert свой собсвенный токенайзер, который разбивает слова на мелкие токены, поэтому нам нужно корректно сопоставить токены и соответсвующие им неры.

In [10]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [11]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/5137 [00:00<?, ? examples/s]

Map:   0%|          | 0/1285 [00:00<?, ? examples/s]

#### Сохраняем словарик соотвествия тега и его индекса внутри модели

In [12]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

data_collator = DataCollatorForTokenClassification(tokenizer)

Some weights of the model checkpoint at xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
model

XLMRobertaForTokenClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_featu

In [14]:
model.classifier = torch.nn.Linear(1024, len(label_list), bias=True).cuda()
model.num_labels = len(label_list)

In [15]:
model

XLMRobertaForTokenClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_featu

### В качестве метрик возьмем precision, recall, accuracy, для этого воспользуемся специализированной под Ner задачу библиотеку seqeval

In [16]:
metric = load_metric("seqeval")

/tmp/ipykernel_76521/152412463.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


### Обучение

In [47]:
args = TrainingArguments(
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    output_dir="models/xlm_roberta"
)

In [48]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [49]:
# trainer = Trainer(
#     model,
#     args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["test"],
#     data_collator=data_collator,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics
# )

# trainer.evaluate()

In [50]:
# from transformers.trainer import logger as noisy_logger
# noisy_logger.setLevel(logging.WARNING)

In [51]:
# Для дообучения берта можно эксперементировать с заморозкой/разморозкой разных слоев, здесь мы оставим все слои размороженными 
# Для быстроты обучения можно заморозить всю бертовую часть, кроме классификатора, но тогда качесвто будет похуже
# for param in model.parameters():
#     param.requires_grad = True

In [52]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [53]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.312765,0.534657,0.619244,0.573850,0.911907
2,No log,0.303146,0.530329,0.597361,0.561853,0.909396
3,No log,0.315811,0.521158,0.639558,0.574320,0.909185


KeyboardInterrupt: 

In [ ]:
model.save_pretrained("models/xlm_roberta/")

In [25]:
trainer.evaluate()

{'eval_loss': 0.40144357085227966,
 'eval_precision': 0.5195944744175016,
 'eval_recall': 0.5724737082761774,
 'eval_f1': 0.5447538538040775,
 'eval_accuracy': 0.905239093160602}

### Тест

In [ ]:
# Посчитаем метрики на отложенном датасете

predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

In [ ]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

### Сохранение модели

### Загрузка модели

## New Data

In [1]:
# import os
import json
import numpy as np
import pandas as pd
import razdel
from tqdm.autonotebook import tqdm

import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from datasets import Dataset, DatasetDict

from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import load_dataset, load_metric

# import logging

/tmp/ipykernel_1222036/1682736822.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


---

## Preparing data

In [2]:
data = pd.read_csv("ner_data_train.csv")
data.head(5)

,video_info,entities
0,<НАЗВАНИЕ:> Агент 117: Из Африки с любовью — Р...,"{""label"":""локация""\,""offset"":26\,""length"":6\,""..."
1,<НАЗВАНИЕ:> Коленвал Инфинити Ку икс 56= 5.6 V...,"{""label"":""организация""\,""offset"":196\,""length""..."
2,<НАЗВАНИЕ:> ВЫЗОВ ДЕМОНА = Вызвал Серого Челов...,"{""label"":""название проекта""\,""offset"":12\,""len..."
3,<НАЗВАНИЕ:> Довоенная немецкая кирха в Калинин...,"{""label"":""не найдено""\,""offset"":162\,""length"":..."
4,"<НАЗВАНИЕ:> ""Спартаку"" помогли судьи? Локомоти...","{""label"":""команда""\,""offset"":13\,""length"":8\,""..."


In [3]:
# данные спарсены с Толоки, поэтому могут иметь проблемы с символами и их нужно избежать, 
# удалить лишние '\' например, преобразовать из str в список dict-ов
df = data.copy()
df['entities'] = df['entities'].apply(lambda l: l.replace('\,', ',')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: l.replace('\\\\', '\\')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: '[' + l + ']'if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: json.loads(l)if isinstance(l, str) else l)

df.head(5)

,video_info,entities
0,<НАЗВАНИЕ:> Агент 117: Из Африки с любовью — Р...,"[{'label': 'локация', 'offset': 26, 'length': ..."
1,<НАЗВАНИЕ:> Коленвал Инфинити Ку икс 56= 5.6 V...,"[{'label': 'организация', 'offset': 196, 'leng..."
2,<НАЗВАНИЕ:> ВЫЗОВ ДЕМОНА = Вызвал Серого Челов...,"[{'label': 'название проекта', 'offset': 12, '..."
3,<НАЗВАНИЕ:> Довоенная немецкая кирха в Калинин...,"[{'label': 'не найдено', 'offset': 162, 'lengt..."
4,"<НАЗВАНИЕ:> ""Спартаку"" помогли судьи? Локомоти...","[{'label': 'команда', 'offset': 13, 'length': ..."


In [4]:
# Теперь из наших данных нам нужно извлечь для каждого слова (токена) его
# тег (label) из разметки, чтобы потом предать в модель классификации токенов

def extract_labels(item):
    
    # воспользуемся удобным токенайзером из библиотеки razdel, 
    # она помимо разбиения на слова, сохраняет важные
    # для нас числа - начало и конец слова в токенах
    
    raw_toks = list(razdel.tokenize(item['video_info']))
    words = [tok.text for tok in raw_toks]
    # присвоим для начала каждому слову тег 'О' - тег, означающий отсутствие NER-а
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item['video_info'])
    # так как NER можем состаять из нескольких слов, то нам нужно сохранить эту инфорцию
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    labels = item['entities']
    if isinstance(labels, dict):
        labels = [labels]
    if labels is not None:
        for e in labels:
            if e['label'] != 'не найдено':
                e_words = sorted({idx for idx in char2word[e['offset']:e['offset']+e['length']] if idx is not None})
                if e_words:
                    word_labels[e_words[0]] = 'B-' + e['label']
                    for idx in e_words[1:]:
                        word_labels[idx] = 'I-' + e['label']
                else:
                    continue
            else:
                continue
        return {'tokens': words, 'tags': word_labels}
    else: return {'tokens': words, 'tags': word_labels}

In [5]:
# extract_labels(df.iloc[0])

### Making train ant test

In [6]:
ner_data = [extract_labels(item) for i, item in df.iterrows()]
ner_train, ner_test = train_test_split(ner_data, test_size=0.2, random_state=1)

In [7]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O',
 'B-Дата',
 'B-бренд',
 'B-вид спорта',
 'B-видеоигра',
 'B-команда',
 'B-лига',
 'B-локация',
 'B-модель',
 'B-название проекта',
 'B-организация',
 'B-персона',
 'B-сезон',
 'B-серия',
 'I-Дата',
 'I-бренд',
 'I-вид спорта',
 'I-видеоигра',
 'I-команда',
 'I-лига',
 'I-локация',
 'I-модель',
 'I-название проекта',
 'I-организация',
 'I-персона',
 'I-сезон',
 'I-серия']

In [8]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 6357
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 65
    })
})

---

## Модель

In [9]:
model_checkpoint = "xlm-roberta-large"
batch_size = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, device=device)
device

device(type='cuda')

#### У Bert свой собсвенный токенайзер, который разбивает слова на мелкие токены, поэтому нам нужно корректно сопоставить токены и соответсвующие им неры.

In [10]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [11]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/5137 [00:00<?, ? examples/s]

Map:   0%|          | 0/1285 [00:00<?, ? examples/s]

#### Сохраняем словарик соотвествия тега и его индекса внутри модели

In [12]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

data_collator = DataCollatorForTokenClassification(tokenizer)

Some weights of the model checkpoint at xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
model

XLMRobertaForTokenClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_featu

In [14]:
model.classifier = torch.nn.Linear(1024, len(label_list), bias=True).cuda()
model.num_labels = len(label_list)

In [15]:
model

XLMRobertaForTokenClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_featu

### В качестве метрик возьмем precision, recall, accuracy, для этого воспользуемся специализированной под Ner задачу библиотеку seqeval

In [62]:
metric = load_metric("seqeval")

### Обучение

In [63]:
args = TrainingArguments(
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=7,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    output_dir="models/xlm_roberta"
)

In [64]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [65]:
# trainer = Trainer(
#     model,
#     args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["test"],
#     data_collator=data_collator,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics
# )

# trainer.evaluate()

In [66]:
# from transformers.trainer import logger as noisy_logger
# noisy_logger.setLevel(logging.WARNING)

In [67]:
# Для дообучения берта можно эксперементировать с заморозкой/разморозкой разных слоев, здесь мы оставим все слои размороженными 
# Для быстроты обучения можно заморозить всю бертовую часть, кроме классификатора, но тогда качесвто будет похуже
# for param in model.parameters():
#     param.requires_grad = True

In [68]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [69]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.332525,0.525924,0.627474,0.572229,0.909663


TrainOutput(global_step=161, training_loss=0.198312332911521, metrics={'train_runtime': 282.9288, 'train_samples_per_second': 18.157, 'train_steps_per_second': 0.569, 'total_flos': 3367775178141534.0, 'train_loss': 0.198312332911521, 'epoch': 1.0})

In [ ]:
model.save_pretrained("models/xlm_roberta/")

In [ ]:
trainer.evaluate()

{'eval_loss': 0.40144357085227966,
 'eval_precision': 0.5195944744175016,
 'eval_recall': 0.5724737082761774,
 'eval_f1': 0.5447538538040775,
 'eval_accuracy': 0.905239093160602}

### Тест

In [ ]:
# Посчитаем метрики на отложенном датасете

predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

In [ ]:
model.save_pretrained()

In [ ]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

In [70]:
import torch
from transformers import pipeline

pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device='cuda')

def predict_ner(text, tokenizer, model, pipe, verbose=True):
    tokens = tokenizer(text, truncation=True, is_split_into_words=True, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    
    with torch.no_grad():
        pred = model(**tokens)
    # print(pred.logits.shape)
    indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
    token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
    labels = []
    for t, idx in zip(token_text, indices):
        if '##' not in t:
            labels.append(label_list[idx])
        if verbose:
            print(f'{t:15s} {label_list[idx]:10s}')
    return labels

### submission

In [71]:
from tqdm.notebook import tqdm

import pandas as pd

from razdel import tokenize

In [72]:
test = pd.read_csv("ner_data_test.csv")
test.head(5)

,video_info,entities_prediction
0,"<НАЗВАНИЕ:> БОЕЦ БЕЗ ПРАВИЛ, ТРЕЙЛЕР на русско...",NaN
1,<НАЗВАНИЕ:> ШОК! КАК ЖЕ ЭТОМУ БОМЖУ ВЕЗЕТ В Br...,NaN
2,<НАЗВАНИЕ:> ДРЕВНИЕ РОБОТЫ NATASHA TALON И LEO...,NaN
3,<НАЗВАНИЕ:> ЖЕНА ПУТЕШЕСТВЕННИКА ВО ВРЕМЕНИ = ...,NaN
4,<НАЗВАНИЕ:> Кинозал ДК приглашает на мультфиль...,NaN


In [73]:
test.shape

(1606, 2)

In [74]:
# данные спарсены с Толоки, поэтому могут иметь проблемы с символами и их нужно избежать, 
# удалить лишние '\' например, преобразовать из str в список dict-ов
# test = data.copy()
# test['entities'] = test['entities'].apply(lambda l: l.replace('\,', ',')if isinstance(l, str) else l)
# test['entities'] = test['entities'].apply(lambda l: l.replace('\\\\', '\\')if isinstance(l, str) else l)
# test['entities'] = test['entities'].apply(lambda l: '[' + l + ']'if isinstance(l, str) else l)
# test['entities'] = test['entities'].apply(lambda l: json.loads(l)if isinstance(l, str) else l)

# test.head(5)

In [75]:

# submission = pd.DataFrame(columns=[['video_info', 'entities_prediction']])
# submission['entities_prediction'] = submission['entities_prediction'].astype('object')

# def sample_submission(text, tokenizer, model, pipe, submission):
#     for i, elem in tqdm(enumerate(ner_test.iloc[:100])):
#         labels = predict_ner(elem['tokens'], tokenizer, model, pipe, verbose=False)
#         submission.loc[i, 'video_info'] = elem["tokens"]

#         submission.loc[i, 'entities_prediction'] = [[label] for label in labels]
#     return submission

In [76]:
# elem

In [77]:
test = test.astype(object)

In [78]:
sample = pd.read_csv("sample_submission.csv")
sample_len = sample["entities_prediction"].apply(lambda x: len(eval(x)))

for i, elem in tqdm(enumerate(test["video_info"]), total=len(test["video_info"])):
    raw_toks = list(tokenize(elem))
    words = [tok.text for tok in raw_toks]
    
    labels = predict_ner(words, tokenizer, model, pipe, verbose=False)
    # submission.loc[i, 'video_info'] = elem["tokens"]
    
    labels = labels[1:-1]
        
    if len(labels) < sample_len[i]:
        amount = sample_len[i] - len(labels)
        labels += ['O'] * amount
        print(amount)
    
    labels = labels[:sample_len[i]]
    
    if len(labels) != sample_len[i]:
        print("что")
        # labels = labels[:len(words)]

    # test.loc[i, 'entities_prediction'] = [[label] for label in labels]
    test.loc[i, 'entities_prediction'] = [[label] for label in labels]
# return submission

  0%|          | 0/1606 [00:00<?, ?it/s]

In [79]:
len(labels), len(words)

(112, 112)

In [80]:
len([tok.text for tok in list(tokenize(test["video_info"][0]))])

157

In [81]:
len(test.loc[0]["entities_prediction"])

157

In [91]:
print("\n".join([f"{text}\t\t{ent}" for (text, ent) in 
                 zip([tok.text for tok in list(tokenize(test["video_info"][0]))], 
                     test.loc[0]["entities_prediction"])]))

<		O
НАЗВАНИЕ		O
:		O
>		O
БОЕЦ		O
БЕЗ		O
ПРАВИЛ		O
,		B-название проекта
ТРЕЙЛЕР		B-название проекта
на		B-название проекта
русском		I-название проекта
,		I-название проекта
фильм		I-название проекта
2021		O
|		O
боевик		O
,		O
MMA		O
<		O
ОПИСАНИЕ		O
:		O
>		O
Больше		O
информации		O
в		B-Дата
нашей		O
группе		O
ВК		O
:		O
<		O
LINK		O
>		O
Русский		B-вид спорта
трейлер		B-вид спорта
фильма		O
БОЕЦ		O
БЕЗ		O
ПРАВИЛ		O
2021		O
г		O
🎬		O
Боец		O
без		O
правил		O
(		O
2021		O
)		O
русский		O
трейлер		O
Notorious		O
Nick		O
(		O
2021		O
)		O
/		O
США		O
драма		O
,		O
боевик		O
Дата		O
выхода		B-название проекта
:		B-название проекта
7		B-название проекта
августа		I-название проекта
2021		I-название проекта
г		I-название проекта
(		B-Дата
мир		I-Дата
)		O
2		O
сентября		B-название проекта
2021		B-название проекта
г		I-название проекта
(		I-название проекта
Россия		O
)		B-Дата
Ник		O
Ньюэлл		O
,		O
однорукий		O
боец		O
ММА		B-название проекта
,		B-название проекта
получает		B-название прое

In [83]:
test["entities_prediction"]

0       [O, O, O, O, O, O, O, B-название проекта, B-на...
1       [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
2       [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
3       [O, O, O, O, O, O, O, B-название проекта, B-на...
4       [O, O, O, O, O, O, O, B-организация, B-организ...
                              ...                        
1601    [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
1602    [O, O, O, O, O, O, O, B-персона, B-персона, I-...
1603    [O, O, O, O, O, O, O, O, O, O, O, B-персона, I...
1604    [O, O, O, O, O, O, O, B-персона, I-персона, I-...
1605    [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
Name: entities_prediction, Length: 1606, dtype: object

In [84]:
print(test["entities_prediction"].apply(len).tolist())

[157, 165, 150, 64, 110, 116, 131, 69, 135, 125, 62, 116, 82, 82, 95, 192, 138, 126, 71, 86, 44, 57, 91, 83, 92, 153, 92, 122, 89, 170, 84, 61, 65, 113, 125, 96, 115, 82, 70, 120, 182, 174, 139, 124, 183, 65, 151, 79, 155, 178, 117, 147, 99, 56, 154, 68, 179, 81, 46, 79, 69, 93, 110, 56, 140, 123, 117, 136, 132, 83, 126, 151, 95, 75, 68, 74, 152, 141, 125, 61, 63, 110, 107, 48, 120, 82, 84, 84, 65, 65, 142, 85, 120, 206, 87, 53, 124, 181, 173, 50, 162, 91, 162, 83, 93, 169, 53, 245, 213, 100, 81, 110, 151, 76, 132, 97, 64, 159, 108, 77, 78, 164, 94, 110, 148, 121, 103, 65, 54, 116, 190, 152, 182, 182, 87, 152, 80, 103, 104, 49, 44, 128, 79, 80, 93, 170, 127, 77, 99, 115, 115, 111, 165, 146, 79, 70, 51, 74, 187, 71, 88, 143, 65, 78, 93, 67, 172, 94, 135, 97, 92, 147, 70, 114, 118, 51, 164, 155, 65, 63, 77, 116, 66, 69, 67, 157, 75, 121, 92, 141, 185, 183, 104, 89, 111, 111, 61, 63, 151, 75, 106, 111, 90, 86, 68, 173, 88, 175, 188, 176, 62, 93, 59, 113, 80, 55, 94, 71, 79, 130, 181, 110,

In [85]:
test.astype(object)

,video_info,entities_prediction
0,"<НАЗВАНИЕ:> БОЕЦ БЕЗ ПРАВИЛ, ТРЕЙЛЕР на русско...","[O, O, O, O, O, O, O, B-название проекта, B-на..."
1,<НАЗВАНИЕ:> ШОК! КАК ЖЕ ЭТОМУ БОМЖУ ВЕЗЕТ В Br...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,<НАЗВАНИЕ:> ДРЕВНИЕ РОБОТЫ NATASHA TALON И LEO...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,<НАЗВАНИЕ:> ЖЕНА ПУТЕШЕСТВЕННИКА ВО ВРЕМЕНИ = ...,"[O, O, O, O, O, O, O, B-название проекта, B-на..."
4,<НАЗВАНИЕ:> Кинозал ДК приглашает на мультфиль...,"[O, O, O, O, O, O, O, B-организация, B-организ..."
...,...,...
1601,"<НАЗВАНИЕ:> Милейшие детеныши животных, которы...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1602,<НАЗВАНИЕ:> Диана Шурыгина набросилась на зрит...,"[O, O, O, O, O, O, O, B-персона, B-персона, I-..."
1603,<НАЗВАНИЕ:> Губернатор Евгений Куйвашев и непо...,"[O, O, O, O, O, O, O, O, O, O, O, B-персона, I..."
1604,<НАЗВАНИЕ:> Александр Горелов = о вакцинации о...,"[O, O, O, O, O, O, O, B-персона, I-персона, I-..."


In [86]:
test.to_csv("submit_xlm_roberta_v1.csv", index=False)

In [87]:
pd.read_csv("submit_xlm_roberta_v1.csv")

,video_info,entities_prediction
0,"<НАЗВАНИЕ:> БОЕЦ БЕЗ ПРАВИЛ, ТРЕЙЛЕР на русско...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-названи..."
1,<НАЗВАНИЕ:> ШОК! КАК ЖЕ ЭТОМУ БОМЖУ ВЕЗЕТ В Br...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,<НАЗВАНИЕ:> ДРЕВНИЕ РОБОТЫ NATASHA TALON И LEO...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,<НАЗВАНИЕ:> ЖЕНА ПУТЕШЕСТВЕННИКА ВО ВРЕМЕНИ = ...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-названи..."
4,<НАЗВАНИЕ:> Кинозал ДК приглашает на мультфиль...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-организ..."
...,...,...
1601,"<НАЗВАНИЕ:> Милейшие детеныши животных, которы...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1602,<НАЗВАНИЕ:> Диана Шурыгина набросилась на зрит...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-персона..."
1603,<НАЗВАНИЕ:> Губернатор Евгений Куйвашев и непо...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1604,<НАЗВАНИЕ:> Александр Горелов = о вакцинации о...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-персона..."


In [89]:
pd.read_csv("sample_submission.csv")

,video_info,entities_prediction
0,"<НАЗВАНИЕ:> БОЕЦ БЕЗ ПРАВИЛ, ТРЕЙЛЕР на русско...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,<НАЗВАНИЕ:> ШОК! КАК ЖЕ ЭТОМУ БОМЖУ ВЕЗЕТ В Br...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,<НАЗВАНИЕ:> ДРЕВНИЕ РОБОТЫ NATASHA TALON И LEO...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,<НАЗВАНИЕ:> ЖЕНА ПУТЕШЕСТВЕННИКА ВО ВРЕМЕНИ = ...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,<НАЗВАНИЕ:> Кинозал ДК приглашает на мультфиль...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
...,...,...
1601,"<НАЗВАНИЕ:> Милейшие детеныши животных, которы...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1602,<НАЗВАНИЕ:> Диана Шурыгина набросилась на зрит...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1603,<НАЗВАНИЕ:> Губернатор Евгений Куйвашев и непо...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1604,<НАЗВАНИЕ:> Александр Горелов = о вакцинации о...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
